**Aluno**: Airton Silva Monteiro


Lista de Exercícios: Teste de Funções

In [2]:
import numpy as np
import time
from scipy.optimize import minimize
from scipy.optimize import approx_fprime

Exercício 1 - Função 105 - Rosenbrock


$f_{105}(\mathbf{x}) = \sum_{i=1}^{D-1} \left[ 100(x_{i+1} - x_i^2)^2 + (x_i - 1)^2 \right]$



In [3]:
import numpy as np

def rosenbrock(x):
    D = len(x)

    f_rbk = sum(100 * (x[i+1] - x[i]**2)**2 + (x[i] - 1)**2 for i in range(D - 1))

    grad = np.zeros_like(x)
    for i in range(D - 1):
        xi = x[i]
        xi1 = x[i + 1]
        diff = xi1 - xi**2

        grad[i] += -400 * xi * diff + 2 * (xi - 1)

        grad[i + 1] += 200 * diff

    hess = np.zeros((D, D))
    for i in range(D - 1):
        xi = x[i]

        # Diagonal entries
        hess[i, i] += 1200 * xi**2 - 400 * x[i + 1] + 2
        hess[i + 1, i + 1] += 200

        # Off-diagonal (symmetric)
        hess[i, i + 1] += -400 * xi
        hess[i + 1, i] += -400 * xi

    

    return f_rbk, grad, hess


Avaliando a função em $X = \begin{pmatrix}
1\\
1
\end{pmatrix}$

In [ ]:

x = np.array([1.0, 1.0])
fval, grad, hess = rosenbrock(x)

print("Function value:", fval)     
print("Gradient:\n", grad)         
print("Hessian:\n", hess) 

Function value: 0.0
Gradient:
 [0. 0.]
Hessian:
 [[ 802. -400.]
 [-400.  200.]]


In [7]:
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: rosenbrock(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
}

dimensions = [2, 5, 10, 20, 50]
all_results = {}

for D in dimensions:
    print(f"\n====== Teste para D = {D} ======\n")
    x0 = np.random.uniform(-2, 2, D)  # Random starting point
    results = {}

    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: rosenbrock(x)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000  # ms

        results[name] = {
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "nhev": res.get("nhev", None),
            "time": elapsed_time,
            "success": res.success
        }

    all_results[D] = results

    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")



====== Teste para D = 2 ======

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   5.8735e-10 |    167 |    N/A |    N/A |       6.99 |     True
BFGS                 |   4.0769e-20 |     54 |     54 |    N/A |       7.00 |     True
Newton-CG            |   1.0364e-09 |     50 |     50 |     34 |       4.99 |     True
trust-ncg            |   3.6473e-12 |     36 |     32 |     31 |       4.00 |     True
trust-krylov         |   2.3807e-10 |     24 |     24 |     21 |      25.00 |     True
trust-exact          |   2.9407e-14 |     21 |     18 |     21 |       6.11 |     True

====== Teste para D = 5 ======

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   1.3878e-09 |    873 |    N/A |    N/A |

**Exercíco 2**:  Função 114 - Scahffer 3


$f_{114}(\mathbf{x}) = 0.5 + \frac{
    \sin^2\left( \cos\left( \left|x_1^2 - x_2^2\right| \right) \right) - 0.5
}{
    \left(1 + 0.001(x_1^2 + x_2^2)\right)^2
}
$

In [19]:
def schaffer_n3(x):
    assert len(x) == 2
    
    x1, x2 = x[0], x[1]
    a = x1**2 - x2**2
    A = np.abs(x1**2 - x2**2)
    B = np.cos(A)
    C = np.sin(B)**2 - 0.5
    D = (1 + 0.001 * (x1**2 + x2**2))**2
    f_sha = 0.5 + C / D
    
    #Derivada do Numerador:
    dC_dA = 2 * np.sin(B) * np.cos(B) * (-np.sin(A))

    sign_a = 1 if a >= 0 else -1
    dA_dx1 = sign_a * 2 * x1
    dA_dx2 = sign_a * -2 * x2

    dC_dx1 = dC_dA * dA_dx1
    dC_dx2 = dC_dA * dA_dx2


    #Derivada do Denominador:
    dD_dx1 = 4 * 0.001 * (1 + 0.001 * (x1**2 + x2**2)) * x1
    dD_dx2 = 4 * 0.001 * (1 + 0.001 * (x1**2 + x2**2)) * x2

    #Aplicando a regra da cadeia
    df_dx1 = (dC_dx1 * D - C * dD_dx1) / D**2
    df_dx2 = (dC_dx2 * D - C * dD_dx2) / D**2
    grad = np.array([df_dx1, df_dx2])

    #Calculo de Matriz Hessiana
    # Computando a derivada de C em relação a A
    d2C_dA2 = (
        2 * np.cos(B)**2 * np.cos(A)**2
        - 2 * np.sin(B)**2 * np.cos(A)**2
        - 2 * np.sin(B) * np.cos(B) * np.sin(A)
    )

    #Derivada segunda de A
    d2A_dx1x1 = sign_a * 2
    d2A_dx2x2 = sign_a * -2
    d2A_dx1x2 = 0  # Derivative of abs function is not cross-coupled

    #Derivada segunda de C em relação x
    d2C_dx1x1 = d2C_dA2 * dA_dx1**2 + dC_dA * d2A_dx1x1
    d2C_dx2x2 = d2C_dA2 * dA_dx2**2 + dC_dA * d2A_dx2x2
    d2C_dx1x2 = d2C_dA2 * dA_dx1 * dA_dx2

    #Derivada segunda de D
    common = (1 + 0.001 * (x1**2 + x2**2))
    d2D_dx1x1 = 4 * 0.001 * (0.001 * 2 * x1**2 + common)
    d2D_dx2x2 = 4 * 0.001 * (0.001 * 2 * x2**2 + common)
    d2D_dx1x2 = 4 * 0.001 * (0.001 * 2 * x1 * x2)

    h11 = ((d2C_dx1x1 * D - 2 * dC_dx1 * dD_dx1 - C * d2D_dx1x1) * D + 2 * C * dD_dx1**2) / D**3
    h22 = ((d2C_dx2x2 * D - 2 * dC_dx2 * dD_dx2 - C * d2D_dx2x2) * D + 2 * C * dD_dx2**2) / D**3
    h12 = ((d2C_dx1x2 * D - dC_dx1 * dD_dx2 - dC_dx2 * dD_dx1 - C * d2D_dx1x2) * D + 2 * C * dD_dx1 * dD_dx2) / D**3

    hess = np.array([[h11, h12], [h12, h22]])



    return f_sha, grad, hess


In [ ]:
#Avaliando para ponto do mínimo global da função:
x = np.array([0.0, 1.253115])
fval, grad = schaffer_n3(x)

print("f(x):", fval)
print("Gradient:", grad)

f(x): 0.0015668545260126288
Gradient: [0.00000000e+00 4.61847774e-07]


In [20]:
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: schaffer_n3(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: schaffer_n3(x)[1], "hess": lambda x: schaffer_n3(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: schaffer_n3(x)[1], "hess": lambda x: schaffer_n3(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: schaffer_n3(x)[1], "hess": lambda x: schaffer_n3(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: schaffer_n3(x)[1], "hess": lambda x: schaffer_n3(x)[2]},
}

initial_points = [
    np.array([10.0, 10.0]),
    np.array([-50.0, 50.0]),
    np.array([0.0, 1.253115]),
    np.array([30.0, -30.0]),
    np.array([-1.0, 2.0])
]

for x0 in initial_points:
    print(f"\n==== Ponto Inicial x0 = {x0} ====\n")

    results = {}
    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: schaffer_n3(x)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000

        results[name] = {
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "nhev": res.get("nhev", None),
            "time": elapsed_time,
            "success": res.success
        }
    
    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.6e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")




==== Ponto Inicial x0 = [10. 10.] ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          | 7.764534e-03 |    400 |    N/A |    N/A |      19.01 |    False
BFGS                 | 5.003376e-01 |     16 |     16 |    N/A |       2.00 |     True
Newton-CG            | 6.444954e-01 |      1 |      1 |      1 |       0.00 |     True
trust-ncg            | 5.020839e-01 |      9 |      9 |      8 |       0.98 |     True
trust-krylov         | 5.020788e-01 |     10 |     10 |      9 |      11.00 |     True
trust-exact          | 1.525493e-01 |    401 |    392 |    401 |      85.09 |    False

==== Ponto Inicial x0 = [-50.  50.] ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          | 3.810621e-01 |    400 |

**Exercício 3**:Função 114 - Scahffer 3


$f_{142}(\mathbf{x}) = \sum_{i=1}^{D-1} \left(x_{i+1}^2 + x_i^2\right)^{0.25} 
\left[\sin^2\left(50 \cdot \left(x_{i+1}^2 + x_i^2\right)^{0.1}\right) + 0.1\right]$

In [21]:

import numpy as np

def stretched_v_sine_wave(x):
    D = len(x)
    f_svs = 0.0
    grad = np.zeros(D)
    hess = np.zeros((D, D))

    for i in range(D - 1):
        xi, xi1 = x[i], x[i + 1]
        s = xi**2 + xi1**2
        s_025 = s**0.25
        s_01 = s**0.1
        sin_term = np.sin(50 * s_01)
        sin2_term = sin_term**2
        cos_term = np.cos(50 * s_01)

        f_svs += s_025 * (sin2_term + 0.1)

        #Gradiente

        d_s = np.array([2 * xi, 2 * xi1])               
        d_s_025 = 0.25 * s**(-0.75) * d_s               
        d_s_01 = 0.1 * s**(-0.9) * d_s                  
        d_sin2 = 2 * sin_term * cos_term * 50 * d_s_01  

        grad_i = d_s_025 * (sin2_term + 0.1) + s_025 * d_sin2

        grad[i] += grad_i[0]
        grad[i + 1] += grad_i[1]

        #Matriz Hessiana

        d2_s = np.array([[2, 0], [0, 2]])

        def d2_s_pow(a):
            term1 = a * (a - 1) * s**(a - 2) * np.outer(d_s, d_s)
            term2 = a * s**(a - 1) * d2_s
            return term1 + term2

        d2_s_025 = d2_s_pow(0.25)
        d2_s_01 = d2_s_pow(0.1)

        d2_sin2 = (
            2 * (cos_term**2 - sin_term**2) * (50**2) * np.outer(d_s_01, d_s_01)
            + 2 * sin_term * cos_term * 50 * d2_s_01
        )

        h_block = (
            d2_s_025 * (sin2_term + 0.1)
            + np.outer(d_s_025, d_sin2) + np.outer(d_sin2, d_s_025)
            + s_025 * d2_sin2
        )
        
        hess[i, i] += h_block[0, 0]
        hess[i, i + 1] += h_block[0, 1]
        hess[i + 1, i] += h_block[1, 0]
        hess[i + 1, i + 1] += h_block[1, 1]

    return f_svs, grad, hess


In [24]:
x = np.array([1.0, 2.0])
fval, grad, hess = stretched_v_sine_wave(x)

print("f(x):", fval, "\n")
print("Gradiente:", grad, "\n")
print("Matriz Hessianda:\n", hess)

f(x): 1.1520606282110522 

Gradiente: [-3.18735045 -6.3747009 ] 

Matriz Hessianda:
 [[ -8.3195189  -10.26433691]
 [-10.26433691 -23.71602427]]


In [25]:
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: stretched_v_sine_wave(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: stretched_v_sine_wave(x)[1], "hess": lambda x: stretched_v_sine_wave(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: stretched_v_sine_wave(x)[1], "hess": lambda x: stretched_v_sine_wave(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: stretched_v_sine_wave(x)[1], "hess": lambda x: stretched_v_sine_wave(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: stretched_v_sine_wave(x)[1], "hess": lambda x: stretched_v_sine_wave(x)[2]},
}


dimensions = [2, 5, 10, 20, 50]

for D in dimensions:
    print(f"\n==== Testando para D = {D} ====\n")

    x0 = np.random.uniform(-5, 5, D)
    results = {}

    for name, opts in methods.items():
        start_time = time.time()
        res = minimize(lambda x: stretched_v_sine_wave(x)[0], x0, **opts, options={"disp": False})
        elapsed_time = (time.time() - start_time) * 1000

        results[name] = {
            "x*": res.x,
            "fval": res.fun,
            "nfev": res.nfev,
            "njev": res.get("njev", None),
            "nhev": res.get("nhev", None),
            "time": elapsed_time,
            "success": res.success
        }
        
    header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
    print(header)
    print("-" * len(header))
    for method, data in results.items():
        print(f"{method:<20} | {data['fval']:12.6e} | {data['nfev']:6d} | "
              f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
              f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
              f"{data['time']:10.2f} | {str(data['success']):>8}")
    







==== Testando para D = 2 ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          | 2.510477e-01 |     86 |    N/A |    N/A |       9.79 |     True
BFGS                 | 2.510477e-01 |      8 |      8 |    N/A |       2.01 |     True
Newton-CG            | 2.510477e-01 |      6 |      6 |      4 |       2.00 |     True
trust-ncg            | 2.510477e-01 |      7 |      6 |      5 |       3.00 |     True
trust-krylov         | 2.510477e-01 |      7 |      7 |      5 |       6.96 |     True
trust-exact          | 2.510477e-01 |      8 |      6 |      8 |       3.00 |     True

==== Testando para D = 5 ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          | 6.867512e-01 |    172 |    N/A |    N/A |  

**Exercício 4**: Função 61 - Hansen

$f_{61}(\mathbf{x}) = 
\sum_{i=0}^{4} (i + 1)\cos\left((i + 1)x_1 + i + 1\right) +
\sum_{j=0}^{4} (j + 1)\cos\left((j + 1)(x_2 + 1) + j + 1\right)$



In [26]:
import numpy as np

def hansen_function(x):
    x1, x2 = x[0], x[1]

    f = 0.0
    grad = np.zeros(2)
    hess = np.zeros((2, 2))

    for i in range(5):
        a = (i + 1)
        arg = a * x1 + a
        cos_term = np.cos(arg)
        sin_term = np.sin(arg)

        f += a * cos_term
        grad[0] += -a**2 * sin_term
        hess[0, 0] += -a**3 * cos_term

    for j in range(5):
        b = (j + 1)
        arg = b * (x2 + 1) + b
        cos_term = np.cos(arg)
        sin_term = np.sin(arg)

        f += b * cos_term
        grad[1] += -b**2 * sin_term
        hess[1, 1] += -b**3 * cos_term

    return f, grad, hess


In [ ]:
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: hansen_function(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: hansen_function(x)[1], "hess": lambda x: hansen_function(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: hansen_function(x)[1], "hess": lambda x: hansen_function(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: hansen_function(x)[1], "hess": lambda x: hansen_function(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: hansen_function(x)[1], "hess": lambda x: hansen_function(x)[2]},
}

In [30]:
start_points = [
    np.array([-7.5, -7.7]),
    np.array([4.9, -1.4])
]

results = []

for name, opts in methods.items():
    print(f"\n Método: {name}")
    res = minimize(lambda x: hansen_function(x)[0], x0, **opts, options={"disp": False})
    print(f"  x* = {res.x}")
    print(f"  f(x*) = {res.fun}")


 Método: Nelder-Mead
  x* = [ 4.53510353 -1.25195609]
  f(x*) = -18.148949144398692

 Método: BFGS
  x* = [ 4.85194346 -1.38626956]
  f(x*) = -7.737510121457003

 Método: Newton-CG
  x* = [ 4.85052657 -1.38586473]
  f(x*) = -7.816630043980013

 Método: trust-ncg
  x* = [ 4.85052657 -1.38586473]
  f(x*) = -7.816630043980013

 Método: trust-krylov
  x* = [ 4.85052657 -1.38586473]
  f(x*) = -7.816630043980013

 Método: trust-exact
  x* = [ 4.86873873 -1.32609053]
  f(x*) = -8.08252678407457
